In [ ]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ee
import geemap

# ee.Authenticate()
# ee.Initialize()

### Create an interactive map

In [ ]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Add data to the map

In [ ]:
point = ee.Geometry.Point([-80.55, 43])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2020-01-01', '2020-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

### Check image properties

In [ ]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2020-06-16'

In [ ]:
image.get('CLOUD_COVER').getInfo()

0.47

### import AAFC label dataset



In [ ]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

use the [Canada AAFC Annual Crop Inventory](https://developers.google.com/earth-engine/datasets/catalog/AAFC_ACI#bands) to create label dataset for training


![](https://www.researchgate.net/profile/Meisam-Amani/publication/345082789/figure/fig1/AS:952723008651265@1604158274050/Annual-Space-based-Crop-Inventory-ACI-produced-by-Agriculture-and-Agri-Food-Canada.png)


In [ ]:
# nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())

affc = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date('2020-01-01', '2020-12-31')).first().clip(image.geometry())

Map.addLayer(affc, {}, 'AAFC')
Map

Map(center=[43, -80.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

## Prepare for consecutive class labels

In [ ]:
raw_class_values = affc.get('landcover_class_values').getInfo()

class_palette = affc.get('landcover_class_palette').getInfo()

value_palette_dictionary = dict(zip(raw_class_values, class_palette))

raw_class_values

[10,
 20,
 30,
 34,
 35,
 50,
 80,
 85,
 110,
 120,
 122,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 160,
 161,
 162,
 163,
 167,
 168,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 210,
 220,
 230]

In [ ]:
class_palette


['000000',
 '3333ff',
 '996666',
 'cc6699',
 'e1e1e1',
 'ffff00',
 '993399',
 '501b50',
 'cccc00',
 'cc6600',
 'ffcc33',
 '7899f6',
 'ff9900',
 '660000',
 'dae31d',
 'd6cc00',
 'd2db25',
 'd1d52b',
 'cace32',
 'c3c63a',
 'b9bc44',
 'a7b34d',
 'b9c64e',
 '999900',
 'e9e2b1',
 '92a55b',
 '809769',
 'ffff99',
 '98887c',
 '799b93',
 '5ea263',
 '52ae77',
 '41bf7a',
 'd6ff70',
 '8c8cff',
 'd6cc00',
 'ff7f00',
 '315491',
 'cc9933',
 '896e43',
 '996633',
 '8f6c3d',
 'b6a472',
 '82654a',
 'a39069',
 'b85900',
 'b74b15',
 'ff8a8a',
 'ffcccc',
 '6f55ca',
 'ffccff',
 'dc5424',
 'd05a30',
 'd20000',
 'cc0000',
 'dc3200',
 'ff6666',
 'c5453b',
 '7442bd',
 'ffcccc',
 'b5fb05',
 'ccff05',
 '07f98c',
 '00ffcc',
 'cc33cc',
 '8e7672',
 'b1954f',
 '749a66',
 '009900',
 '006600',
 '00cc00',
 'cc9900']

In [ ]:
initial_train_accuracy = [[0],
 [0.8996450467892869],
 [0.00819672131147541],
 [0.17928902627511592],
 [0],
 [0.1495766698024459],
 [0.1269968051118211],
 [0],
 [0],
 [0],
 [0.6437947494033412],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.055793991416309016],
 [0],
 [0.13953488372093023],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.13537117903930132],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.2768595041322314],
 [0.6237767306995288],
 [0.7143840330350998]]

In [ ]:
reduced_class_values = raw_class_values

for i in range(len(class_palette)):
  if initial_train_accuracy[i][0] == 0:
    reduced_class_values[i] = 0

reduced_class_values = [i for i in reduced_class_values if i != 0]


In [ ]:
veg_classes = [145, 147, 158, 210, 220, 230]
veg_palette = []

for x in veg_classes:
  veg_palette.append(value_palette_dictionary[x])
new_palette = ['3333ff'] +  veg_palette
new_palette

['3333ff', '92a55b', 'ffff99', 'cc9933', '006600', '00cc00', 'cc9900']

In [ ]:
reduced_class_values

[20, 30, 34, 50, 80, 122, 145, 147, 158, 210, 220, 230]

In [ ]:
to_group_reduced_class_values = reduced_class_values
for i in range(len(reduced_class_values)):
  if not to_group_reduced_class_values[i] in veg_classes:
    to_group_reduced_class_values[i] = 20


In [ ]:
to_group_reduced_class_values

[20, 20, 20, 20, 20, 20, 145, 147, 158, 210, 220, 230]

In [ ]:
affc = affc.remap(reduced_class_values, to_group_reduced_class_values).select(
    ['remapped'], ['landcover']
)

In [ ]:
grouped_reduced_class_values = list(dict.fromkeys(to_group_reduced_class_values))


In [ ]:
n_classes = len(grouped_reduced_class_values)
new_class_values = list(range(0, n_classes))

In [ ]:
new_class_values

[0, 1, 2, 3, 4, 5, 6]

In [ ]:
affc = affc.remap(grouped_reduced_class_values, new_class_values).select(
    ['remapped'], ['landcover']
)
affc = affc.set('landcover_class_values', new_class_values)
affc = affc.set('landcover_class_palette', new_palette)

In [ ]:
affc.get('landcover_class_values').getInfo()

[0, 1, 2, 3, 4, 5, 6]

## Make  dataset

In [ ]:
# Make the training dataset.
points = affc.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 15000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)


## Split training and validation

In [ ]:
points = points.randomColumn();

training_points = points.filter('random <= 0.7');
validation_points = points.filter('random > 0.7');

Map.addLayer(training_points, {}, 'training', False)
Map.addLayer(validation_points, {}, 'validation', False)

In [ ]:
print(training_points.size().getInfo())

6453


In [ ]:
print(training_points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-79.93783236408851, 42.70212558121815]}, 'id': '0', 'properties': {'landcover': 0, 'random': 0.5720078189015817}}


### Train the classifier

In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{'collection': training_points, 'properties': [label], 'scale': 30}
)

validation = image.select(bands).sampleRegions(
    **{'collection': validation_points, 'properties': [label], 'scale': 30}
)

# Train a CART classifier with default parameters.
trainedClassifier = ee.Classifier.smileRandomForest(27).train(training, label, bands)

In [ ]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 319, 'B2': 324, 'B3': 273, 'B4': 149, 'B5': 116, 'B6': 93, 'B7': 76, 'landcover': 0}}


# Accuracy assessment
## Training dataset

In [ ]:
# Get a confusion matrix and overall accuracy for the training sample.
train_accuracy = trainedClassifier.confusionMatrix();
# trainAccuracy.getInfo()

###Training - Overall Accuracy

In [ ]:
train_accuracy.accuracy().getInfo()

0.9872868217054264

###Training -  Kappa

In [ ]:
train_accuracy.kappa().getInfo()

0.9829565107772276

###Training - Producer's Accuracy

In [ ]:
train_accuracy.producersAccuracy().getInfo()

[[1],
 [0.9826086956521739],
 [0.986046511627907],
 [0.9663366336633663],
 [0.8571428571428571],
 [0.9943661971830986],
 [0.9343434343434344]]

###Training - Comsumer's Accuracy

In [ ]:
train_accuracy.consumersAccuracy().getInfo()


[[1,
  0.9741379310344828,
  0.9747126436781609,
  0.9858585858585859,
  1,
  0.9737931034482759,
  0.9946236559139785]]

# Validation dataset

In [ ]:
validated = validation.classify(trainedClassifier)

In [ ]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [ ]:
# test_accuracy.getInfo()

## Validation - Overall Accuracy


In [ ]:
test_accuracy.accuracy().getInfo()

0.8099581908019764

## Validation - Kappa

In [ ]:
test_accuracy.kappa().getInfo()

0.7455581965752964

## Validation - Producer's Accuracy

In [ ]:
test_accuracy.producersAccuracy().getInfo()

[[0.9990662931839402],
 [0.7918552036199095],
 [0.7238095238095238],
 [0.5247524752475248],
 [0.2],
 [0.8253968253968254],
 [0.3764705882352941]]

## Validation - Consumers’ Accuracy

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

[[0.9981343283582089,
  0.7231404958677686,
  0.6654991243432574,
  0.6074498567335244,
  0.3333333333333333,
  0.7831325301204819,
  0.5423728813559322]]

### Classify the image

In [ ]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trainedClassifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

Map(center=[43, -80.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

### Render categorical map

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps. 

In [ ]:
class_values = affc.get('landcover_class_values').getInfo()
print(class_values)

[0, 1, 2, 3, 4, 5, 6]


In [ ]:
class_palette = affc.get('landcover_class_palette').getInfo()
print(class_palette)

['3333ff', '92a55b', 'ffff99', 'cc9933', '006600', '00cc00', 'cc9900']


In [ ]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[43, -80.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

### Visualize the result

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Add a legend to the map

In [ ]:
# Map.add_legend(builtin_legend='NLCD')
# Map

### Export the result

Export the result directly to your computer:

In [ ]:
# import os

# out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# out_file = os.path.join(out_dir, 'landcover.tif')

In [ ]:
# geemap.ee_export_image(landcover, filename=out_file, scale=900)

Export the result to Google Drive:

In [ ]:
# geemap.ee_export_image_to_drive(
#     landcover, description='landcover', folder='export', scale=900
# )